In [1]:
import kaggle
import pandas as pd
import numpy as np
import zipfile
import statsmodels.api as sm

from sklearn.neighbors import KDTree
from sklearn.preprocessing import StandardScaler

In [10]:
# !kaggle competitions download -c titanic


  0%|          | 0.00/34.1k [00:00<?, ?B/s]
100%|██████████| 34.1k/34.1k [00:00<00:00, 719kB/s]


In [17]:
# with zipfile.ZipFile('titanic.zip', 'r') as zip_ref:
#     zip_ref.extractall()

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.describe()
# 177 Fehlende Werte bei 'Age' (20%)
# 2 Fehlende Werte bei 'Embarked' (<1%)

# Spalte isFemale hinzugefügt
df['isFemale'] = df.apply(lambda x: x['Sex']=='female', axis=1)
df.drop('Sex', axis=1, inplace=True)

# Für die zwei fehlenden Werte den Modus eingesetzt
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
one_hot = pd.get_dummies(df['Embarked'])
one_hot.rename(columns={'S':'Embarked_S', 'C':'Embarked_C', 'Q':'Embarked_Q'}, inplace=True)
df.drop('Embarked',axis = 1, inplace=True)
df = df.join(one_hot)


# Dummies für die Klasse
one_hot = pd.get_dummies(df['Pclass'])
one_hot.rename(columns={1:'Class_1', 2:'Class_2', 3:'Class_3'}, inplace=True)
df.drop('Pclass',axis = 1, inplace=True)
df = df.join(one_hot)

# Ich vermute eine quadratische Beziehung zwischen Age und Survival
df['AgeSq'] = df.apply(lambda x: x['Age']**2, axis=1)


# Dummy-Variablen für Mr, Miss, Mrs und Master
title = df.apply(lambda x: x['Name'].split(',')[1].split('.')[0].lstrip(), axis=1)

df['isMr'] = title.apply(lambda x: x == 'Mr')
df['isMiss'] = title.apply(lambda x: x == 'Miss')
df['isMrs'] = title.apply(lambda x: x == 'Mrs')
df['isMaster'] = title.apply(lambda x: x == 'Master')



In [15]:
mask = df['Age'].isna()

exog = ['SibSp', 'Fare', 'Embarked_S', 'Embarked_C', 'Embarked_Q', 'isFemale', 
    'Class_1', 'Class_2', 'Class_3', 'isMiss', 'isMr', 'isMrs', 'isMaster']

df_std = df[exog]

col_names = df_std.columns
scaler = StandardScaler().fit(df_std.values)
df_std = scaler.transform(df_std.values)
df_std = pd.DataFrame(df_std, columns=col_names)

df_ageMissing = df_std[mask]
df_age = df_std[~mask]
df_age.insert(0, 'Age', df[~mask]['Age'])

df_age.reset_index(drop=True, inplace=True)
# df_ageMissing.reset_index(drop=True, inplace=True)


In [16]:
tree = KDTree(df_age[exog].values, leaf_size=2)
imputed_values_knn_1 = []

for index, entry in enumerate(df_ageMissing.values):

    dist, ind = tree.query([entry], k=1)
    ind = ind[0][0]
    imputed_values_knn_1.append(df_age['Age'][ind])


In [17]:
imputed_values_knn_1

[70.5,
 34.0,
 45.0,
 45.5,
 19.0,
 28.0,
 40.0,
 16.0,
 28.5,
 33.0,
 35.0,
 40.0,
 16.0,
 20.0,
 28.0,
 30.0,
 12.0,
 28.0,
 35.0,
 16.0,
 35.0,
 35.0,
 28.0,
 26.0,
 15.0,
 35.0,
 70.5,
 9.0,
 29.0,
 59.0,
 17.0,
 11.0,
 62.0,
 64.0,
 4.0,
 16.0,
 23.0,
 31.0,
 32.0,
 70.5,
 16.0,
 14.0,
 40.0,
 28.0,
 9.0,
 23.0,
 14.5,
 15.0,
 59.0,
 56.0,
 70.5,
 16.0,
 55.0,
 16.0,
 29.0,
 64.0,
 30.0,
 55.0,
 16.0,
 40.0,
 30.0,
 35.0,
 30.0,
 14.0,
 15.0,
 36.0,
 28.0,
 26.0,
 45.0,
 45.5,
 19.0,
 19.0,
 40.0,
 45.0,
 16.0,
 52.0,
 28.0,
 21.0,
 9.0,
 28.0,
 19.0,
 29.0,
 31.0,
 33.0,
 59.0,
 70.5,
 26.0,
 19.0,
 18.0,
 35.0,
 35.0,
 70.5,
 35.0,
 29.0,
 21.0,
 59.0,
 47.0,
 29.0,
 9.0,
 18.0,
 11.0,
 40.5,
 16.0,
 45.0,
 35.0,
 65.0,
 45.5,
 28.5,
 64.0,
 28.5,
 24.0,
 40.5,
 36.0,
 70.5,
 27.0,
 70.5,
 35.0,
 26.0,
 28.5,
 16.0,
 15.0,
 33.0,
 35.0,
 16.0,
 6.0,
 45.5,
 28.0,
 47.0,
 20.0,
 15.0,
 70.5,
 21.0,
 40.0,
 30.0,
 32.0,
 42.0,
 28.0,
 19.0,
 28.0,
 26.0,
 35.0,
 29.0,
 15.0,
 32.0